In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import heapq


from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn import cross_validation, metrics   #Additional scklearn functions

from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')

In [ ]:
test = pd.read_csv('../input/test.csv')

# One Hot - Encoding

can we do it?! yes?


In [ ]:
# split dat train data
X, Y = train_test_split(train, test_size=0.2, random_state=42)
# train_x, test_x = train_test_split(X, test_size=0.33, random_state=42)
# train_y, test_y = train_test_split(Y, test_size=0.33, random_state=42)

In [ ]:
# didn't really work
# one_hot_encoded_training_predictors = pd.get_dummies(train_x)

In [ ]:
def feature_filler(df):
    features = df.columns.values.tolist()
    features.remove('Id')
    diagnostics = df.copy()
    new_df = df.copy()

    missing = {}

    for feature in features:
        feature_percentage = diagnostics[feature].isnull().sum()/len(diagnostics[feature])
        if feature_percentage == 0:
            pass
        else:
            missing[feature] = feature_percentage

    missing_features = [k for k,v in missing.items() if v != 0]
    
    possible_values = {}

    for feature in missing_features:
        total = diagnostics[feature].value_counts().sum()
        possible_values[feature] = diagnostics[feature].value_counts()/total

    for each_key in possible_values.keys():
        probability = heapq.nlargest(10,possible_values[each_key])
        normal_probability = [x / sum(probability) for x in probability]

        values = possible_values[each_key][possible_values[each_key].isin(probability)].index.tolist()[:10]
        new_column = new_df[new_df[each_key].isnull()][each_key].apply(lambda x: np.random.choice(values,p=normal_probability))

        new_df.loc[new_df[each_key].isnull(),each_key] = new_column
    
    return new_df

In [ ]:
# hashing is dumb, does converting everything to bits have the same effect?

def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    # str_result = ''.join(str(x) for x in result)
    # int_result = int(str_result)
    summy = sum(result)
    return summy

def frombits(bits):
    chars = []
    for b in range(len(bits) / 8):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

In [ ]:
def bitify(df):
    
    bit_df = df.copy()
    
    g = df.columns.to_series().groupby(bit_df.dtypes).groups
    dg = {k.name: v for k, v in g.items()}
    
    #bit_df = bit_df.fillna('Nothing')
    bit_df = bit_df.fillna(0)
    
    bit_df[dg['object']] = bit_df[dg['object']].applymap(tobits)
    
    #bit_df.replace('Nothing',0,inplace=True)
    
    return bit_df

In [ ]:
def feature_select(df, estimator, number_features):
    n = number_features
    new_df = df.copy()
    colnames = new_df.columns.tolist()
    colnames.remove('SalePrice')
    estimator.fit(new_df[colnames],new_df['SalePrice'])
    selector = RFE(estimator, n, step=1)
    selector = selector.fit(new_df[colnames], new_df['SalePrice'])
    best_index = np.where(selector.support_)[0]
    
    new_cols = []
    for i in best_index:
        new_cols.append(colnames[i])
    
    return new_cols
    

In [ ]:
# depending on how it is split can use smaller sets

fill_train = feature_filler(X)
fill_test = feature_filler(Y)

In [ ]:
bit_train_x = bitify(fill_train) 
bit_test_x = bitify(fill_test)

In [ ]:
estimator = Ridge(alpha=0.1)

In [ ]:
best_features = feature_select(bit_train_x,estimator,10)

In [ ]:
all_feat = bit_train_x.columns.tolist()
all_feat.remove('SalePrice')

In [ ]:
best_features

In [ ]:
clf = LinearRegression()

In [ ]:
clf.fit(bit_train_x[all_feat],bit_train_x['SalePrice'])
clf.score(bit_test_x[all_feat],bit_test_x['SalePrice'])

Potential Models:

from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# how many features is optimal? Linear Regression (lr)
#lr_feature_score = []
#lr = LinearRegression()
#for i in range(1,(len(all_feat)+1)):
#    best_features = feature_select(bit_train_x,estimator,i)
#    lr.fit(bit_train_x[best_features],bit_train_x['SalePrice'])
#    score = lr.score(bit_test_x[best_features],bit_test_x['SalePrice'])
#    lr_feature_score.append(score)
#print(np.argmax(lr_feature_score))

In [ ]:
# how many features is optimal? Gradient Boosting Regressor (gbr)
#gbr_feature_score = []
#gbr = GradientBoostingRegressor()
#for i in range(1,(len(all_feat)+1)):
#    best_features = feature_select(bit_train_x,estimator,i)
#    gbr.fit(bit_train_x[best_features],bit_train_x['SalePrice'])
#    score = gbr.score(bit_test_x[best_features],bit_test_x['SalePrice'])
#    gbr_feature_score.append(score)
#print(np.argmax(gbr_feature_score))

In [ ]:
# how many features is optimal? Random Forest Regressor (rfr)
#rfr_feature_score = []
#rfr = RandomForestRegressor()
#for i in range(1,(len(all_feat)+1)):
#    best_features = feature_select(bit_train_x,estimator,i)
#    rfr.fit(bit_train_x[best_features],bit_train_x['SalePrice'])
#    score = rfr.score(bit_test_x[best_features],bit_test_x['SalePrice'])
#    rfr_feature_score.append(score)
#print(np.argmax(rfr_feature_score))

In [ ]:
#print(np.argmax(rfr_feature_score))
#print(rfr_feature_score[np.argmax(rfr_feature_score)])

In [ ]:
#print(np.argmax(lr_feature_score))
#print(lr_feature_score[np.argmax(lr_feature_score)])

In [ ]:
#print(np.argmax(gbr_feature_score))
#print(gbr_feature_score[np.argmax(gbr_feature_score)])

In [ ]:
# Gradient Booster is the best default performer

In [ ]:
# compare different estimator
#estimator = Lasso(alpha=0.1)
#all_feat = bit_train_x.columns.tolist()
#gbr_feature_score = []
#gbr = GradientBoostingRegressor()
#for i in range(1,(len(all_feat)+1)):
#    best_features = feature_select(bit_train_x,estimator,i)
#    gbr.fit(bit_train_x[best_features],bit_train_x['SalePrice'])
#    score = gbr.score(bit_test_x[best_features],bit_test_x['SalePrice'])
#    gbr_feature_score.append(score)
#    best_number_features = np.argmax(gbr_feature_score)
#print(best_number_features)

In [ ]:
# performs better with 67 features vs 72
# score of .89 vs .87
#gbr_feature_score[predictors]

In [ ]:
# we have a model and a good number of features, now to test various parameters

def modelfit(alg, dtrain, predictors, performCV=True, printFeatureImportance=True, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['SalePrice'])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain['SalePrice'], cv=cv_folds)
    
    #Print model report:
    print ("\nModel Report")
    print ("R2 Score : %.4g" % metrics.r2_score(dtrain['SalePrice'].values, dtrain_predictions))
    print ("explained_variance_score (Train): %f" % metrics.explained_variance_score(dtrain['SalePrice'], dtrain_predictions))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

In [ ]:
predictors1 = feature_select(bit_train_x,estimator,63)
predictors2 = feature_select(bit_train_x,estimator,73)

In [ ]:
gbm0 = GradientBoostingRegressor(random_state=10)

In [ ]:
modelfit(gbm0, bit_train_x, predictors1)

In [ ]:
modelfit(gbm0, bit_train_x, predictors2)

In [ ]:
submission = test.copy()

In [ ]:

submission = feature_filler(submission)
submission = bitify(submission) 

In [ ]:
submission['SalePrice'] = gbm0.predict(submission[predictors2])

In [ ]:
submission[['Id','SalePrice']].to_csv('submission.csv', index=False)